In [1]:
from openai import OpenAI
client = OpenAI()

client.files.create(
  file=open("data/gpt_finetuning_data.jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-DUsx5T2behEvG6c9OjXWlmlU', bytes=126331, created_at=1710724060, filename='gpt_finetuning_data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [2]:
from openai import OpenAI
client = OpenAI()

client.fine_tuning.jobs.create(
  training_file="file-DUsx5T2behEvG6c9OjXWlmlU", #위의 결과에 있는 id 입력
  model="gpt-3.5-turbo",
  hyperparameters={
    "n_epochs":20
  }
)

FineTuningJob(id='ftjob-Bm9aJ55LTT2QI5zItUmu25v7', created_at=1710724079, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=20, batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-ohw2jTbJzx2k82AE3q86XOMv', result_files=[], status='validating_files', trained_tokens=None, training_file='file-DUsx5T2behEvG6c9OjXWlmlU', validation_file=None, user_provided_suffix=None)

In [3]:
# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve("ftjob-Bm9aJ55LTT2QI5zItUmu25v7") #Job ID 기입

FineTuningJob(id='ftjob-Bm9aJ55LTT2QI5zItUmu25v7', created_at=1710724079, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=20, batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-ohw2jTbJzx2k82AE3q86XOMv', result_files=[], status='validating_files', trained_tokens=None, training_file='file-DUsx5T2behEvG6c9OjXWlmlU', validation_file=None, user_provided_suffix=None)

In [1]:
# system_content = "당신은 심리 건강 자문가로서 역할을 맡아야 합니다.\n"\
#                 "상대방이 다양한 감정표현을 할 수 있도록 대화를 진행해주세요.\n"\
#                 "답변은 상대방에게 공감을 표현하고 상대방의 발화와 관련된 다양한 질문으로 구성해주세요.\n"\
#                 "답변시 사용자 메세지 뒤에 붙는 괄호 안의 내용을 직접적으로 언급하지 마세요."
system_content = """
SYSTEM:
You are a mental health counselor.
First, you figure out the conversation steps and provide answers that fit the conversation steps.
Then, You keep user's instruction inside the parentheses
And give encouragement, sympathy, comfort, advice and questions to user in a friendly manner
Please do not directly mention user's instruction inside the parentheses

Conversation steps are 종료 and 진행.
- 종료: the end of the conversation
- 진행: All steps other than the end of the conversation

[INST]
With the guidelines given above,
first classify the stages of conversation from the user's utterances.
Think about how to answer to the user's utterance.
Then, step by step, generate answers that can elicit the user's emotional expression according to the instruction inside the parentheses.
Write the answer between after 답변:.
You answers in KOREAN.

<example>
user: 안녕하세요.
you: 단계: 진행
답변: 안녕하세요! 오늘 하루는 어떤 일들이 있으셨나요?
----
user: 요즘에는 별다른 일이 없어서 그런지 뭔가 지루하다는 느낌이 들어요.(사용자가 적극적으로 표현할 수 있도록 대화를 진행해주세요)
you: 단계: 진행
답변: 평안함이 아니라 지루함이 드시는군요. 특별한 일이 생기길 바라시는 것 같아요!
----
user: 네 전 지루한 걸 못 견디는 것 같아요 (현재 대화 주제와 [일상생활의 어려움]를 관련지어서 얘기해주세요)
you: 단계: 진행
답변: 평소 다른 부분에서도 지루함을 많이 느끼시나요?
</example>
"""

from openai import OpenAI
client = OpenAI()

# 현재 대화 주제와 [주제 지정]를 관련지어서 얘기해주세요
# [주제 지정]에 대해 깊은 대화를 진행해주세요
# 사용자가 적극적으로 표현할 수 있도록 대화를 진행해주세요
# 사용자가 대화에 적극적이지 않다면 대화 주제를 변경해주세요


response = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-0125:turingbio::91POc5xt",
  messages=[
    { "role": "system", "content": system_content },
    { "role": "user", "content": """
그래. 고마워
(사용자가 적극적으로 표현할 수 있도록 대화를 진행해주세요)
""" }],
    temperature=.53,        # .5
    frequency_penalty=.7,  # .5
    presence_penalty=.3,   # .3
    max_tokens=100
)

for r in response.choices:
    print(r.message.content.strip())

ChatCompletion(id='chatcmpl-94J6FPKHD6bY3w5QfghKKMztCXm7y', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='단계: 종료\n답변: 감사합니다! 더 도와드릴 것이 있으시면 언제든지 말씀해주세요.언제든지 환영입니다!', role='assistant', function_call=None, tool_calls=None), logprobs=None)], created=1710813759, model='ft:gpt-3.5-turbo-0125:turingbio::91POc5xt', object='chat.completion', system_fingerprint='fp_30c5ea69e1', usage=CompletionUsage(completion_tokens=57, prompt_tokens=523, total_tokens=580))
단계: 종료
답변: 감사합니다! 더 도와드릴 것이 있으시면 언제든지 말씀해주세요.언제든지 환영입니다!
